<a href="https://colab.research.google.com/github/yshun0206/Enhancing-Security-in-Next-Generation-URLLC-System-with-RIS-and-Artificial-Noise/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cvxpy as cp
import numpy as np
import time
from scipy.special import erfinv
K = 4   #Number of users
J = 2   # Number of eavesdroppers
N_T = 8 # Number of antennas at BS
N = 4  # Number of time slots
D_k = np.array([2, 4, 4, 4, 4, 4, 4, 4])
### Define cell parameters ###
r1 = 50  # m
r2 = 500  # m
np.random.seed(1111)  # You can replace 0 with any other integer to set a different seed
a = np.log10(np.exp(1))
epsilon = 1e-6
delta = 1e-6
eta = 1.5
beta = 1000
T_f = 0.25 # Frame duration (ms)
num_elements = 16  # Number of RIS elements
W = 1   # Bandwidth (in MHz)
nbar = int((W*1e6 * T_f*10**(-3))/N)
# BS located at center of cell
bs_pos = np.array([0, 0])
RIS_pos = np.array([50, 50])
# Generate URLLC user positions
user_pos = np.random.uniform(low=r1, high=r2, size=(K, 2))
# Generate eavesdropper positions
eaves_pos = np.random.uniform(low=r1, high=r2, size=(J, 2))

# Calculate distances between BS and RIS
BS2RIS_dist = np.linalg.norm(RIS_pos - bs_pos)
# Calculate distances between BS and users/eavesdroppers
user_dists = np.linalg.norm(user_pos - bs_pos, axis=1)
eaves_dists = np.linalg.norm(eaves_pos - bs_pos, axis=1)
# Calculate distances between RIS and users/eavesdroppers
RIS2user_dist = np.linalg.norm(user_pos - RIS_pos, axis=1)
RIS2eaves_dist = np.linalg.norm(eaves_pos - RIS_pos, axis=1)

# Calculate pathloss between BS and users/eavesdroppers
pathloss_BS2user = np.sqrt(10**(-30/10) * user_dists**(-4))
pathloss_BS2eaves = np.sqrt(10**(-30/10) * eaves_dists**(-4))
# Calculate pathloss between RIS and users/eavesdroppers
pathloss_RIS2user = np.sqrt(10**(-30/10) * RIS2user_dist**(-2))
pathloss_RIS2eaves = np.sqrt(10**(-30/10) * RIS2eaves_dist**(-2))
# Calculate pathloss between BS and RIS
pathloss_BS2RIS = np.sqrt(10**(-30/10)*BS2RIS_dist**(-2.2))

Hk = np.zeros((K, N_T, N_T), dtype=np.complex_)
Gj = np.zeros((J, N_T, N_T), dtype=np.complex_)
Hkk = np.zeros((K, num_elements+1, N_T), dtype=np.complex_)
Gjj = np.zeros((J, num_elements+1, N_T), dtype=np.complex_)
initial_phase_real = np.random.randn(num_elements)
initial_phase_imag = np.random.randn(num_elements)
phase_real = cp.Variable(num_elements, value = initial_phase_real)
phase_imag = cp.Variable(num_elements, value = initial_phase_imag)
phase_complex = phase_real.value + 1j * phase_imag.value
ris_matrix = np.diag(phase_complex)
initial_phase_real2 = np.random.randn(num_elements+1, num_elements+1)
initial_phase_imag2 = np.random.randn(num_elements+1, num_elements+1)
phase_real2 = cp.Variable((num_elements+1, num_elements+1), value=initial_phase_real2)
phase_imag2 = cp.Variable((num_elements+1, num_elements+1), value=initial_phase_imag2)
# Iterate through each user
for i in range(K):
    # Create channel vectors for BS to user k and BS to eavesdropper j
    BS2RIS = np.sqrt(0.5) * (np.random.randn(num_elements, N_T) + 1j * np.random.randn(num_elements, N_T))
    RIS2USER = np.sqrt(0.5) * (np.random.randn(num_elements ) + 1j * np.random.randn(num_elements))
    h = RIS2USER @ ris_matrix @ BS2RIS
    h_scaled = h
    Hk[i] = np.outer(h_scaled, np.conj(h_scaled).T)
# Similar operations for eavesdropper
for j in range(J):
    BS2RIS = np.sqrt(0.5) * (np.random.randn(num_elements, N_T) + 1j * np.random.randn(num_elements, N_T))
    RIS2USER = np.sqrt(0.5) * (np.random.randn(num_elements ) + 1j * np.random.randn(num_elements))
    g = RIS2USER @ ris_matrix @ BS2RIS
    g_scaled = g
    Gj[j] = np.outer(g_scaled, np.conj(g_scaled).T)

for i in range(K):
    # Create channel vectors for BS to user k and BS to eavesdropper j
    BS2RIS = np.sqrt(0.5) * (np.random.randn(num_elements, N_T) + 1j * np.random.randn(num_elements, N_T))
    RIS2USER = np.sqrt(0.5) * (np.random.randn(num_elements ) + 1j * np.random.randn(num_elements))
    BS2USER = np.sqrt(0.5) * (np.random.randn(N_T) + 1j * np.random.randn(N_T))
    H_all = np.vstack(((np.diag(RIS2USER) @ BS2RIS), BS2USER ))
    Hkk[i] = H_all

# Similar operations for eavesdropper
for j in range(J):
    BS2RIS = np.sqrt(0.5) * (np.random.randn(num_elements, N_T) + 1j * np.random.randn(num_elements, N_T))
    RIS2USER = np.sqrt(0.5) * (np.random.randn(num_elements ) + 1j * np.random.randn(num_elements))
    BS2USER = np.sqrt(0.5) * (np.random.randn(N_T) + 1j * np.random.randn(N_T))
    G_all = np.vstack(((np.diag(RIS2USER) @ BS2RIS), BS2USER ))
    Gjj[j] = G_all
# Create real and imaginary parts separately
initial_Vn_real = np.random.rand(N_T, N_T)
initial_Vn_imag = np.random.rand(N_T, N_T)
initial_Vn_real = (initial_Vn_real + initial_Vn_real.T)/2
initial_Vn_imag = (initial_Vn_imag + initial_Vn_imag.T)/2
Vn_real = [cp.Variable((N_T, N_T), symmetric=True, value = initial_Vn_real) for n in range(N)]
Vn_imag = [cp.Variable((N_T, N_T), symmetric=True, value = initial_Vn_imag) for n in range(N)]
initial_Wn_real = np.random.rand(N_T, N_T)
initial_Wn_imag = np.random.rand(N_T, N_T)
initial_Wn_real = (initial_Wn_real + initial_Wn_real.T)/2
initial_Wn_imag = (initial_Wn_imag + initial_Wn_imag.T)/2
Wn_real = [[cp.Variable((N_T, N_T), symmetric=True, value = initial_Wn_real) for n in range(N)] for k in range(K)]
Wn_imag = [[cp.Variable((N_T, N_T), symmetric=True, value = initial_Wn_imag) for n in range(N)] for k in range(K)]
theta = [cp.Variable(nonneg = True) for k in range(K)]
Vn = [Vn_real[n] + 1j * Vn_imag[n] for n in range(N)]
Wn = [[Wn_real[k][n] + 1j * Wn_imag[k][n] for n in range(N)] for k in range(K)]
# Define objective function
G = cp.sum([cp.trace(Wn_real[k][n]) for n in range(N) for k in range(K)]) + \
    cp.sum([cp.trace(Vn_real[n]) for n in range(N)]) + beta * cp.sum(theta)
obj = cp.Minimize(G)

eigenvalues, eigenvectors = np.linalg.eig(phase_real2.value + 1j*phase_imag2.value)
term = np.conj(eigenvectors[np.argmax(eigenvalues)].T)@(phase_real2+1j*phase_imag2)@eigenvectors[np.argmax(eigenvalues)]
G2 = np.sum([np.trace(Wn_real[k][n].value) for n in range(N) for k in range(K)]) + \
    cp.sum([cp.trace(Vn_real[n]) for n in range(N)]) + term
objphase = cp.Minimize(G)

tau = [cp.Variable(pos=True) for k in range(K)]
C1a=[]
a_k_i = [[cp.Parameter(value = 10) for n in range(N)] for k in range(K)]
Q_inv_epsilon = np.sqrt(2) * erfinv(1 - 2 * epsilon)
Q_inv_delta = np.sqrt(2) * erfinv(1 - 2 * delta)
ak = [[cp.Variable(pos=True) for n in range(N)]for k in  range(K)]
Rk = [nbar*cp.sum([cp.log(1 + ak[k][n]) for n in range(N)]) for k in range(K)]
Vk = [Q_inv_epsilon*nbar**(0.5)*cp.sum([(1-(1 + a_k_i[k][n])**(-2)) for n in range(N)])**(0.5) for k in range(K)]
Vk_grad = [Q_inv_epsilon*nbar**(0.5)*cp.sum([(1+a_k_i[k][n])**(-2)*(a_k_i[k][n]**2 + 2*a_k_i[k][n])**(-0.5) for n in range(N)]) for k in range(K)]
Vk_tilde = [[Vk[k] - Vk_grad[k]*(ak[k][n] - a_k_i[k][n]) for n in range(N)] for k in range(K)]
for n in range(N):
    for k in range(K):
        C1a += [Rk[k] - Vk_tilde[k][n] - tau[k] + theta[k] >= 0]
bjki = [[[cp.Parameter(value=50) for n in range(N)]for k in  range(K)]for j in range(J)]
bjk = [[[cp.Variable(pos=True) for n in range(N)]for k in  range(K)]for j in range(J)]
C1b=[]
Cjk = [[nbar*cp.sum([cp.log(1+bjki[j][k][n]) for n in range(N)]) + Q_inv_delta*nbar**(0.5)*cp.sum([(1+bjki[j][k][n])**(-2) for n in range(N)])**(0.5) for k in range(K)] for j in range(J)]
Cjk_grad = [[nbar*cp.sum([(1+bjki[j][k][n])**(-1) for n in range(N)])+Q_inv_delta*nbar**(0.5)*cp.sum([(bjki[j][k][n]+1)**(-2)*(bjki[j][k][n]**2 + 2*bjki[j][k][n])**(-0.5) for n in range(N)]) for k in range(K)]for j in range(J)]
Cjk_tilde = [[[Cjk[j][k] - Cjk_grad[j][k]*(bjk[j][k][n] - bjki[j][k][n]) for n in range(N)]for k in range(K)] for j in range(J)]
for n in range(N):
    for k in range(K):
        for j in range(J):
            C1b += [Cjk_tilde[j][k][n] <= tau[k]]
C2=[]
for k in range(K):
    for n in range(D_k[k], N):
        C2 += [cp.trace(Wn_real[k][n]) == 0]

C3 = []
for n in range(N):
    C3 += [Vn_real[n] == Vn_real[n].T]  # Vn_real is a symmetric matrix
    C3 += [Vn_imag[n] == -Vn_imag[n].T]  # Vn_imag a skew-symmetric matrix

    schur_matrix = cp.vstack([
        cp.hstack([Vn_real[n], -Vn_imag[n]]),
        cp.hstack([Vn_imag[n], Vn_real[n]])
    ])  # the Schur complement of Vn

    C3 += [schur_matrix >> 0]  # This adds the constraint that the Schur complement M is PSD
    #     C3 += [Vn[n] >> 0] # This adds the constraint that Vn is PSD
    for i in range(N_T):
        C3 += [Vn_imag[n][i, i] == 0]  # the diagonal elements of Vn_imag are equal to 0
        C3 += [Vn_real[n][i, i] >= 0]  # the diagonal elements of Vn_real are greater than or equal to 0

C4 = []
for k in range(K):
    for n in range(N):
        C4 += [Wn_real[k][n] == Wn_real[k][n].T]  # Wn_real is a symmetric matrix
        C4 += [Wn_imag[k][n] == -Wn_imag[k][n].T]  # Wn_imag is a skew-symmetric matrix
        schur_matrix = cp.vstack([
            cp.hstack([Wn_real[k][n], -Wn_imag[k][n]]),
            cp.hstack([Wn_imag[k][n], Wn_real[k][n]])
        ])  # the Schur complement of Wn

        C4 += [schur_matrix >> 0]  # This adds the constraint that the Schur complement M is PSD
        for i in range(N_T):
            C4 += [Wn_imag[k][n][i, i] == 0]
            C4 += [Wn_real[k][n][i, i] >= 0]

qk = [[cp.Variable(pos = True) for n in range(N)]for k in  range(K)]
zk = [[cp.Variable(pos = True) for n in range(N)]for k in  range(K)]
# ak = [[cp.Variable(pos = True) for n in range(N)]for k in  range(K)]
q_k_i = [[cp.Parameter(value = 10) for n in range(N)] for k in range(K)]
z_k_i = [[cp.Parameter(value = 15) for n in range(N)] for k in range(K)]

C6a=[]
C6b=[]
C6c=[]
for k in range(K):
    for n in range(N):
        HW_product = cp.real(Hk[k])@Wn_real[k][n] - cp.imag(Hk[k])@Wn_imag[k][n]
        for i in range(N_T):
            C6a += [HW_product[i, i] >= 0]
        C6a += [cp.trace(HW_product) >= qk[k][n]**2] #here
        inter = [cp.trace(cp.real(Hk[k])@Wn_real[l][n] - cp.imag(Hk[k])@ Wn_imag[l][n]) for l in range(K) if l != k]
        for i in range(K-1):
            C6a += [inter[i]>=0]
        inter_user_interference = cp.sum(inter)
        HV_producr = cp.real(Hk[k])@Vn_real[n] - cp.imag(Hk[k])@Vn_imag[n]
        for i in range(N_T):
            C6a += [HV_producr[i, i]>=0]
        intra_user_interference = cp.trace(HV_producr)
        C6b += [inter_user_interference + intra_user_interference + 1 <= zk[k][n]]#here
        C6b += [inter_user_interference + intra_user_interference + 1 >= 0]
        C6c += [2*(q_k_i[k][n] / z_k_i[k][n] + 1e-5) * qk[k][n] - (q_k_i[k][n] / z_k_i[k][n] +1e-5)**2 * zk[k][n] >= ak[k][n]]
bjk = [[[cp.Variable(pos=True) for n in range(N)] for k in range(K)] for j in range(J)]
fjk = [[[cp.Variable(pos=True) for n in range(N)] for k in range(K)] for j in range(J)]
f_jk_i = [[[cp.Parameter(value = 50) for n in range(N)] for k in range(K)] for j in range(J)]

C7a=[]
C7b=[]
for j in range(J):
    for k in range(K):
        for n in range(N):
            E_inter = [cp.real(cp.trace(Gj[j] @ (Wn_real[l][n] + 1j * Wn_imag[l][n]))) for l in range(K) if l != k]
            for i in range(K-1):
                C7a += [E_inter[i]>=0]
            E_inter_user_interference = cp.sum(E_inter)
            GV_producr = cp.real(Gj[j])@Vn_real[n] - cp.imag(Gj[j])@Vn_imag[n]
            for i in range(N_T):
                C7a += [GV_producr[i, i]>=0]
            G_intra_user_interference = cp.trace(GV_producr)
            C7a += [cp.bmat([[bjk[j][k][n], fjk[j][k][n]],
                            [fjk[j][k][n], E_inter_user_interference + G_intra_user_interference + 1]]) >> 0]
            GW_product= cp.real(Gj[j]) @ Wn_real[k][n] - cp.imag(Gj[j]) @ Wn_imag[k][n]
            for i in range(N_T):
                C7b += [GW_product[i, i]>=0]
            rhs = cp.trace(GW_product)
            lhs = (f_jk_i[j][k][n])**2 + 2 * f_jk_i[j][k][n] * (fjk[j][k][n] - f_jk_i[j][k][n])
            C7b += [lhs >= rhs]
C8 = []
for k in range(K):
    C8 += [theta[k] >= 0]

C9 = []
lower_bound = -2 * np.pi
upper_bound = 2 * np.pi
C9 += [phase_real2 >= lower_bound, phase_real2 <= upper_bound]
C9 += [phase_imag2 >= lower_bound, phase_imag2 <= upper_bound]
schur_matrix = cp.vstack([
        cp.hstack([phase_real2, -phase_imag2]),
        cp.hstack([phase_imag2, phase_real2])
    ])  # the Schur complement of Vn

C9 += [schur_matrix >> 0]  # This adds the constraint that the Schur complement M is PSD
C6aa=[]
C6bb=[]
C6cc=[]
for k in range(K):
    for n in range(N):
        HW_product = phase_real2 @ np.real(Hkk[k]@(Wn_real[k][n].value+1j*Wn_imag[k][n].value)@np.transpose(Hkk[k]))\
        - phase_imag2 @ np.imag(Hkk[k]@(Wn_real[k][n].value+1j*Wn_imag[k][n].value)@np.transpose(Hkk[k]))
        for i in range(num_elements+1):
            C6aa += [HW_product[i, i] >= 0]
        C6aa += [cp.trace(HW_product) >= qk[k][n]**2] #here
        inter = [phase_real2 @ np.real(Hkk[k]@(Wn_real[l][n].value+1j*Wn_imag[l][n].value)@np.transpose(Hkk[k])) - phase_imag2 @ np.imag(Hkk[k]@(Wn_real[l][n].value+1j*Wn_imag[l][n].value)@np.transpose(Hkk[k])) for l in range(K) if l != k]
#         inter = [cp.trace(cp.real(Hk[k])@Wn_real[l][n] - cp.imag(Hk[k])@ Wn_imag[l][n]) for l in range(K) if l != k]
        for i in range(K-1):
            C6aa += [inter[i]>=0]
        inter_user_interference = cp.sum(inter)
#         HV_producr = cp.real(Hk[k])@Vn_real[n] - cp.imag(Hk[k])@Vn_imag[n]
        HV_producr = phase_real2 @ np.real(Hkk[k]@(Vn_real[n].value + 1j*Vn_imag[n].value)@np.transpose(Hkk[k]))\
            -phase_imag2 @ np.imag(Hkk[k]@(Vn_real[n].value + 1j*Vn_imag[n].value)@np.transpose(Hkk[k]))
        for i in range(num_elements+1):
            C6aa += [HV_producr[i, i]>=0]
        intra_user_interference = cp.trace(HV_producr)
        C6bb += [inter_user_interference + intra_user_interference + 1 <= zk[k][n]]#here
        C6bb += [inter_user_interference + intra_user_interference + 1 >= 0]
        C6cc += [2*(q_k_i[k][n] / z_k_i[k][n] + 1e-5) * qk[k][n] - (q_k_i[k][n] / z_k_i[k][n] +1e-5)**2 * zk[k][n] >= ak[k][n]]
C7aa=[]
C7bb=[]
for j in range(J):
    for k in range(K):
        for n in range(N):
#             E_inter = [cp.real(cp.trace(Gj[j] @ (Wn_real[l][n] + 1j * Wn_imag[l][n]))) for l in range(K) if l != k]
            E_inter = [phase_real2@(np.real(Gjj[j]@(Wn_real[l][n].value + 1j * Wn_imag[l][n].value)@np.transpose(Gjj[j]))) - phase_imag2@(np.imag(Gjj[j]@(Wn_real[l][n].value + 1j * Wn_imag[l][n].value)@np.transpose(Gjj[j])))for l in range(K) if l != k]
            for i in range(K-1):
                C7aa += [E_inter[i]>=0]
            E_inter_user_interference = cp.trace(cp.sum(E_inter))
# #             GV_producr = cp.real(Gj[j])@Vn_real[n] - cp.imag(Gj[j])@Vn_imag[n]
            GV_producr = phase_real2 @ np.real(Gjj[j]@(Vn_real[n].value + 1j*Vn_imag[n].value)@np.transpose(Gjj[j]))\
            -phase_imag2 @ np.imag(Gjj[j]@(Vn_real[n].value + 1j*Vn_imag[n].value)@np.transpose(Gjj[j]))
            for i in range(num_elements+1):
                C7aa += [GV_producr[i, i]>=0]
            G_intra_user_interference = cp.trace(GV_producr)
            C7aa += [cp.bmat([[bjk[j][k][n], fjk[j][k][n]],
                            [fjk[j][k][n], E_inter_user_interference + G_intra_user_interference + 1]]) >> 0]
# #             GW_product = cp.real(Gj[j]) @ Wn_real[k][n] - cp.imag(Gj[j]) @ Wn_imag[k][n]
            GW_product = phase_real2 @ np.real(Gjj[j]@(Wn_real[k][n].value+1j*Wn_imag[k][n].value)@np.transpose(Gjj[j]))\
                        - phase_imag2 @ np.imag(Gjj[j]@(Wn_real[k][n].value+1j*Wn_imag[k][n].value)@np.transpose(Gjj[j]))
            for i in range(num_elements+1):
                C7bb += [GW_product[i, i]>=0]
            rhs = cp.trace(GW_product)
            lhs = (f_jk_i[j][k][n])**2 + 2 * f_jk_i[j][k][n] * (fjk[j][k][n] - f_jk_i[j][k][n])
            C7bb += [lhs >= rhs]
C10 = [cp.trace(phase_real2) == num_elements + 1, cp.sum(phase_real2 - np.eye(num_elements + 1)) == 0]
Wn_real_np = np.array([[np.array(initial_Wn_real) for n in range(N)] for k in range(K)])
Wn_imag_np = np.array([[np.array(initial_Wn_imag) for n in range(N)] for k in range(K)])
eata = 0
for k in range(K):
    for n in range(N):
        matrix_complex = Wn_real_np[k][n] + 1j * Wn_imag_np[k][n]
        trace_value = np.trace(matrix_complex)
        max_eigenvalue = np.max(np.linalg.eigvals(matrix_complex).real)
        eata = eata + (trace_value - max_eigenvalue)

constraint = C1a + C1b+ C2 + C3 + C4 + C6a + C6b + C6c + C7a + C7b + C8
constraint2 = C1a + C1b+ C3 + C6a + C6b + C6c + C7a + C7b  + C10
max_trans_power = []
max_iter = 20
# # Iteratively solve the problem
for iter_num in range(max_iter):
    start = time.time()
    problem = cp.Problem(obj, constraint)
    problem.solve(eps=10e-15)
    end = time.time()
    total_power = np.sum([np.real(np.trace(Wn[k][n].value)) for n in range(N) for k in range(K)]) + \
                  np.sum([np.real(np.trace(Vn[n].value)) for n in range(N)])
    average_power_watts = total_power / N
    average_power_dbm = 10 * np.log10(np.abs(average_power_watts) * 1000)  # Convert from Watts to dBm
    total_power_n_expr = cp.hstack([cp.sum([cp.real(cp.trace(Wn[k][n])) for k in range(K)]) + cp.real(cp.trace(Vn[n])) for n in range(N)])
    max_total_power = cp.max(total_power_n_expr)
    max_total_power_dBm = 10 * (cp.log(cp.abs(max_total_power) * 1000) / cp.log(10.0))
    max_total_power_dBm_value = max_total_power_dBm.value
    max_trans_power.append(max_total_power_dBm_value)
    print(f"======================Iteration {iter_num + 1}======================")
    print("total_power :", total_power )
    print("average_power_watts :", average_power_watts )
    print(f"Maximum transmit power of the BS: {max_total_power_dBm_value} dBm")
    print(f"Average transmit power of the BS: {average_power_dbm} dBm")
    print(problem.value)
    print(problem.status)
    print("time:", end-start)
     # Update the parameters with the current solution
    for k in range(K):
        for n in range(N):
            q_k_i[k][n].value = max(qk[k][n].value, 1e-3)
            z_k_i[k][n].value = max(zk[k][n].value, 1e-3)
            a_k_i[k][n].value = max(ak[k][n].value, 1e-3)
            for j in range(J):
                f_jk_i[j][k][n].value = max(fjk[j][k][n].value, 1e-3)
                bjki[j][k][n].value = max(bjk[j][k][n].value, 1e-3)
    print()
    beta *= 1.5
    if beta>=5000:
        beta = 5000
    constraint2 = C1a + C1b+ C3 + C6a + C6b + C6c + C7a + C7b  + C10
    start = time.time()
    problem2 = cp.Problem(objphase, constraint2)
    problem2.solve(eps=10e-15)
    end = time.time()
    print('----------------------------')
    print("total_power :", total_power )
    print("average_power_watts :", average_power_watts )
    print(f"Maximum transmit power of the BS: {max_total_power_dBm_value} dBm")
    print(f"Average transmit power of the BS: {average_power_dbm} dBm")
    print(problem.value)
    print(problem.status)
    print("time:", end-start)
    print('----------------------------')

    # for k in range(K):
    #   for n in range(N):
    #     q_k_i[k][n].value = max(qk[k][n].value, 1e-3)
    #     z_k_i[k][n].value = max(zk[k][n].value, 1e-3)
    #     a_k_i[k][n].value = max(ak[k][n].value, 1e-3)
    #     for j in range(J):
    #       f_jk_i[j][k][n].value = max(fjk[j][k][n].value, 1e-3)
    #       bjki[j][k][n].value = max(bjk[j][k][n].value, 1e-3)


/usr/local/lib/python3.10/dist-packages/cvxpy/reductions/solvers/solving_chain.py:213: UserWarning: You are solving a parameterized problem that is not DPP. Because the problem is not DPP, subsequent solves will not be faster than the first one. For more information, see the documentation on Discplined Parametrized Programming, at
	https://www.cvxpy.org/tutorial/advanced/index.html#disciplined-parametrized-programming
  warnings.warn(dpp_error_msg)
/usr/local/lib/python3.10/dist-packages/cvxpy/problems/problem.py:1387: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


======================Iteration 1======================
total_power : 0.2515234094733058
average_power_watts : 0.06288085236832645
Maximum transmit power of the BS: 18.129879172439686 dBm
Average transmit power of the BS: 17.985184200598464 dBm
0.6098465151094692
optimal_inaccurate
time: 138.78031945228577

----------------------------
total_power : 0.2515234094733058
average_power_watts : 0.06288085236832645
Maximum transmit power of the BS: 18.129879172439686 dBm
Average transmit power of the BS: 17.985184200598464 dBm
0.6098465151094692
optimal_inaccurate
time: 45.41033148765564
----------------------------
